In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import Polygon, get_coordinates
import sqlalchemy
import osmnx as ox
import shapely
import os

## settings

In [ ]:
local_crs = 3006
graph_crs = 32634

city_abbr = 'sthlm'

file_folder = "directory/ISA_GPKG_20250324"
file_name = 'ISA.gpkg'
layer_name = 'ISA'

write_to_table = True
write_to_file = True
plot = True

In [ ]:
if 'gbg' in city_abbr:
    minx = 294938.44463674916
    miny = 6376587.771166455
    maxx = 337053.8948209431
    maxy = 6418673.5884089535
elif 'sthlm' in city_abbr:
    minx = 655777.9272083798
    miny = 6568493.165786877
    maxx = 683485.4688108079
    maxy = 6593666.9224746805

bbox = gpd.GeoDataFrame(index=[0], crs=local_crs, geometry=[Polygon([
    (minx, miny),
    (minx, maxy),
    (maxx, maxy),
    (maxx, miny)])])

## read network

In [ ]:
edges = gpd.read_file(os.path.join(file_folder, file_name), layer=layer_name, bbox=bbox)

## pre-process into edges and nodes

In [ ]:
edges.rename(columns={'HTHAST':'maxspeed'}, inplace=True)
edges = edges[['RLID', 'maxspeed', 'geometry']]

# set (default) values to necessary edge attributes
edges['osmid'] = edges.index
edges['length'] = edges.geometry.length
edges[['u', 'v', 'key', 'highway']] = None, None, 0, 'unclassified'

edges.to_crs(graph_crs, inplace=True)

In [ ]:
# define nodes as start/end points of any edge
starts = shapely.get_point(edges.geometry, 0)
ends = shapely.get_point(edges.geometry, -1)

nodes = gpd.GeoDataFrame(geometry=pd.concat([starts, ends]), crs=graph_crs)
nodes.drop_duplicates(subset=['geometry'], inplace=True)
nodes.reset_index(drop=True, inplace=True)
nodes['osmid'] = nodes.index

nodes.set_index('osmid', inplace=True)
nodes['x'] = nodes.geometry.x
nodes['y'] = nodes.geometry.y

In [ ]:
def find_to_from_nodes(row):
    start = get_coordinates(shapely.get_point(row.geometry, 0)).tolist()[0]
    end = get_coordinates(shapely.get_point(row.geometry, -1)).tolist()[0]

    start_osmid = nodes[(nodes.geometry.x==start[0]) & (nodes.geometry.y==start[1])].index[0]
    end_osmid = nodes[(nodes.geometry.x==end[0]) & (nodes.geometry.y==end[1])].index[0]
    return start_osmid, end_osmid

In [ ]:
# define start- and end node id to every edge
edges[['u', 'v']] = edges.apply(lambda row: find_to_from_nodes(row), axis=1, result_type='expand')

In [ ]:
# add a reverse version for all edges, allowing 2-way flows
edges_r = edges.copy()

edges_r.rename(columns={'u':'u_old', 'v':'v_old'}, inplace=True)

edges_r['u'] = edges_r['v_old']
edges_r['v'] = edges_r['u_old']
edges_r['geometry'] = edges_r['geometry'].reverse()

edges_r.drop(columns={'u_old', 'v_old'}, inplace=True)

edges = pd.concat([edges, edges_r])

In [ ]:
def set_key_for_duplicates(df):
    df['key'] = df.groupby(['u', 'v']).cumcount()
    return df

In [ ]:
# in case of multiple edges between the same pair of nodes: keep index unique
edges = set_key_for_duplicates(edges)
edges.set_index(['u', 'v', 'key'], inplace=True)

In [ ]:
# mark curved lines with unique osmid, they need to be maintained as is, others can be simplified
edges['n_coords'] = edges.geometry.count_coordinates()
edges['straight_line'] = edges.apply(lambda x: x.osmid if x.n_coords>2 else -1, axis=1)
edges.drop(columns=['n_coords'], inplace=True)

In [ ]:
# create and simplify the graph
G = ox.graph_from_gdfs(nodes, edges)
G = ox.simplify_graph(G, edge_attrs_differ=['straight_line'])

# get final nodes and edges gdfs after simplifying
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [ ]:
if plot:
    ox.plot_graph(G)

## write to table

In [ ]:
nodes.reset_index(drop=False, inplace=True)
nodes.to_crs(local_crs, inplace=True)

In [ ]:
edges.reset_index(drop=False, inplace=True)
edges.to_crs(local_crs, inplace=True)

In [ ]:
# write edges and nodes to database
if write_to_table:

    url_flowsense = sqlalchemy.URL.create(
        "postgresql+psycopg", port=5432,
        host="host", database="database", username="username")
    engine_flowsense = sqlalchemy.create_engine(url_flowsense)

    nodes.to_postgis(
        name='trafikverket_nodes_{}'.format(city_abbr),
        con=engine_flowsense,
        schema='road_network',
        if_exists='replace',
        index=False)

    edges.to_postgis(
        name='trafikverket_edges_{}'.format(city_abbr),
        con=engine_flowsense,
        schema='road_network',
        if_exists='replace',
        index=False)